In [ ]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_capability())

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/colab_ds.zip" /content/

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/df_train.pkl" /content/
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/df_test.pkl" /content/
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/df_val.pkl" /content/
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/loadbm.py" /content/

In [ ]:
!unzip colab_ds.zip

In [ ]:
import numpy as np
import os
import ntpath
import platform
import matplotlib.pyplot as plt
from loadbm import create_tf_dataset, prepare_for_training

df_train = np.load('df_train.pkl',allow_pickle=True)
df_test = np.load('df_test.pkl',allow_pickle=True)
df_val = np.load('df_val.pkl',allow_pickle=True)

def fix_path(fpath):
  path, file_ = ntpath.split(fpath)
  _, folder = ntpath.split(path)
  
  return os.path.join('colab_ds',folder,file_)

df_train['path'] = df_train['path'].apply(fix_path)
df_test['path'] = df_test['path'].apply(fix_path)
df_val['path'] = df_val['path'].apply(fix_path)

from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_val = shuffle(df_val)

df_train.head()

,path,label
289229,colab_ds/Simuliidae358/0-Simuliidae358.1.2016-...,37
185806,colab_ds/Limnius_vol143/0-Limnius_vol143.1.201...,22
263018,colab_ds/Polycentropus_fla30/0-Polycentropus_f...,31
69461,colab_ds/Elmis_aen632/0-Elmis_aen632.1.2016-12...,8
238615,colab_ds/Oulimnius_tub171/0-Oulimnius_tub171.1...,28


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMSIZE = (224,224,3)
BATCH_SIZE = 32

train_ds = create_tf_dataset(df_train, imsize=IMSIZE, onehot=True)

val_ds = create_tf_dataset(df_val, imsize=IMSIZE, onehot=True)


train_ds = prepare_for_training(train_ds, 
                                shuffle_buffer_size=1000,
                                batch_size=BATCH_SIZE)

val_ds = prepare_for_training(val_ds, 
                              shuffle_buffer_size=1000,
                              batch_size=BATCH_SIZE)

for image, label in train_ds.take(5):
    print(image.shape)
    print(label.shape)


(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


def get_pretrained(imsize=(224, 224, 3), classes=39):
        base_model = InceptionV3(input_shape = imsize, 
                                 weights='imagenet', 
                                 include_top=False)
        
        base_model.trainable = True
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

model = get_pretrained()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics=['accuracy'])


87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.callbacks import CSVLogger
import datetime

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

csv_logger = CSVLogger('26-01-2020_colab.log',append=True)

tr_steps = len(df_train)//BATCH_SIZE
val_steps = len(df_val)//BATCH_SIZE

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 30,
          validation_steps = val_steps,
          callbacks=[csv_logger, tensorboard_callback])

Train for 502 steps, validate for 71 steps
Epoch 1/30
502/502 [==============================] - 99s 197ms/step - loss: 1.7955 - accuracy: 0.4689 - val_loss: 1.9889 - val_accuracy: 0.4384
Epoch 2/30
502/502 [==============================] - 84s 167ms/step - loss: 0.9743 - accuracy: 0.6844 - val_loss: 2.3569 - val_accuracy: 0.4723
Epoch 3/30
502/502 [==============================] - 84s 167ms/step - loss: 0.6896 - accuracy: 0.7773 - val_loss: 1.3844 - val_accuracy: 0.5920
Epoch 4/30
502/502 [==============================] - 84s 166ms/step - loss: 0.5233 - accuracy: 0.8309 - val_loss: 1.0240 - val_accuracy: 0.7020
Epoch 5/30
502/502 [==============================] - 84s 167ms/step - loss: 0.4015 - accuracy: 0.8659 - val_loss: 0.8184 - val_accuracy: 0.7491
Epoch 6/30
502/502 [==============================] - 84s 167ms/step - loss: 0.3320 - accuracy: 0.8881 - val_loss: 1.2692 - val_accuracy: 0.6580
Epoch 7/30
502/502 [==============================] - 84s 166ms/step - loss: 0.2839 - a